# Seventh assignment
The assignment provides problems regarding basic applications of linear algebra and machine learning. Use this notebook to perform the computations and insert your comments into it. All the coding must be done in Python 3. 

The assignment has to be submitted individually!

All your plots have to be labelled properly!  (Non-labeled plots will result in point deductions!)

**Non running cells/tasks will not be considered!**

The tasks/questions are 1.1), 1.2), 1.3), 1.4), 1.5), 2.1), 2.2), 2.3), 2.4), 3.1), 3.2), 3.3) and 3.4). After each task description there is an answer cell for your code or text. For coding tasks they look like this:

```
# YOUR CODE HERE
```
(You can delete the "YOUR CODE HERE" comment, if you like.)

For markdown (text) cells the response cell will include this:

**WRITE YOUR ANSWER HERE**

Sometimes they are followed by test cells. You can run the test cells after you finished the task. If these cells don't show any errors, your answer is right. You can't/shouldn't edit test cells!

After you finished the notebook you can hit the "Validate" button on the top of the notebook to see if all test are good. You can also use the "Validate" button on the "Assignment" tap on the main page of python.ldv.ei.tum.de. Some tasks will be graded manually (e.g. plots, text answers). They don't have a following test cell. 

Please make sure to hit the "Submit" button on the "Assignment" tab on the main page of python.ldv.ei.tum.de before the deadline passes. You will get your final score after the deadline.

**Some tips:**
- Only change cells with `# YOUR CODE HERE` or **WRITE YOUR ANSWER HERE**
- Do not change cell types or the notebook name.
- Do not add other .ipynb files into the ami22 folder or subfolders.
- Do not override the original files in the ami22 folder or subfolders.
- For every plot makes sure that axes are correctly labeled with original labels and not encoded ones. 
- If you are asked to provide multiple plots, make sure that the titles are clear. A plot should be self explanatory, we should not have to look at your code to know what is plotted. 

#### Date of submission: June 28, 2022, 23:55 hrs

In [1]:
import numpy as np
import scipy.signal as sig
# import tensorflow as tf
import tensorflow as tf
from keras.applications.mobilenet_v2 import MobileNetV2, decode_predictions, preprocess_input
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import img_to_array
from keras.datasets import cifar10
from keras.callbacks import EarlyStopping
from keras import utils
from keras.utils.vis_utils import plot_model
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from PIL import Image
import matplotlib.pyplot as plt

In the first part you will be using the famous CIFAR-10 dataset. The dataset consists of 32 by 32 RGB images of 10 different classes. You can assume it being the small sister of ImageNet. We will sketch how you could run a gridsearch using sklearn together with keras. Due to the limited compute capabilites, we will use the training set only and define our own split. 

In [307]:
(X, y), (_,_) = cifar10.load_data() #due to the limited computing 
#capacity we take the training samples only and do our own split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.5, random_state=42)
print(X_train.shape)
# convert the images to floats between 0 and 1
X_train,X_test = X_train.astype('float32')/255, X_test.astype('float32')/255
print(X_test.shape)
# convert the labels to one-hot encoding
(y_train, y_test)=  utils.to_categorical(y_train,10), utils.to_categorical(y_test,10)
# mean center the images
X_train_mean = np.expand_dims(X_train.mean(axis=0),axis=0)
X_train = X_train-X_train_mean
X_test = X_test-X_train_mean


(25000, 32, 32, 3)
(25000, 32, 32, 3)


In [308]:
# Creating the model
def create_cnn_model(n_filters=(32,32), kernel_sizes=((5,5),(3,3)), activations=('relu','relu'), randseed=42):
    tf.random.set_seed(randseed)
    cnn_model = tf.keras.models.Sequential([
    Conv2D(n_filters[0], kernel_sizes[0], activation=activations[0], input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(n_filters[1], kernel_sizes[1], activation=activations[1]),
    Flatten(),
    Dense(10, activation='softmax')
    ],
    name="CNN"
    )
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # cnn_model.summary()
    return cnn_model

model = create_cnn_model()
# utils.plot_model(model, 'img1.jpg')

**Task 1.1**

a) Set up an `EarlyStopping()` callback to monitor the validation loss. Set the `mode=min`, a patience of 3 epochs and make sure the best weights will be restored. Use a verbosity level of 1 and store the callback function to the variable `es`! 

b) Set up a `KerasClassifier()` with `create_cnn_model` as build function. Set the following parameters:
- `validation_split=.25`
- `epochs=25`
- `batch_size=128`
- `verbose=0`
Store the Classifier to the variable `cnn`!

c) Define a dict `param_grid` to test the number of filters `(16,16)` and `(16,32)` as well as the kernel sizes of `(3,3),(3,3)` and `(3,3),(5,5)`!

Finally use the provided code to run the grid search. This can take a few minutes.

In [323]:
# YOUR CODE HERE

es = [EarlyStopping(monitor='val_loss', patience=3, mode='min',verbose=1,restore_best_weights=True)]

cnn = KerasClassifier(model=create_cnn_model, validation_split=.25, epochs=25, batch_size=128,verbose=0,n_filters=(32,32), kernel_sizes=((5,5),(3,3)),randseed=42)
# print(cnn.get_params().keys())
param_grid = dict(n_filters=[(16,16),(16,32)],kernel_sizes=[((3,3),(3,3)),((3,3),(5,5))])
grid1 = GridSearchCV(estimator=cnn, param_grid=param_grid,cv=2,verbose=2)
grid1.fit(X_train,y_train,callbacks=es)
grid1.cv_results_

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] END ..kernel_sizes=((3, 3), (3, 3)), n_filters=(16, 16); total time= 1.0min
[CV] END ..kernel_sizes=((3, 3), (3, 3)), n_filters=(16, 16); total time= 1.3min
[CV] END ..kernel_sizes=((3, 3), (3, 3)), n_filters=(16, 32); total time= 1.3min
[CV] END ..kernel_sizes=((3, 3), (3, 3)), n_filters=(16, 32); total time= 1.3min
[CV] END ..kernel_sizes=((3, 3), (5, 5)), n_filters=(16, 16); total time= 1.6min
[CV] END ..kernel_sizes=((3, 3), (5, 5)), n_filters=(16, 16); total time= 1.5min
[CV] END ..kernel_sizes=((3, 3), (5, 5)), n_filters=(16, 32); total time= 1.6min
[CV] END ..kernel_sizes=((3, 3), (5, 5)), n_filters=(16, 32); total time= 1.4min


{'mean_fit_time': array([67.80751181, 77.94866681, 90.49314749, 85.70162082]),
 'std_fit_time': array([8.21300507, 0.26594853, 1.18730342, 6.00179935]),
 'mean_score_time': array([1.25613296, 1.43620527, 1.99201345, 1.84658766]),
 'std_score_time': array([0.18154633, 0.04441988, 0.01171303, 0.18153882]),
 'param_kernel_sizes': masked_array(data=[((3, 3), (3, 3)), ((3, 3), (3, 3)), ((3, 3), (5, 5)),
                    ((3, 3), (5, 5))],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_filters': masked_array(data=[(16, 16), (16, 32), (16, 16), (16, 32)],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'kernel_sizes': ((3, 3), (3, 3)), 'n_filters': (16, 16)},
  {'kernel_sizes': ((3, 3), (3, 3)), 'n_filters': (16, 32)},
  {'kernel_sizes': ((3, 3), (5, 5)), 'n_filters': (16, 16)},
  {'kernel_sizes': ((3, 3), (5, 5)), 'n_filters': (16, 32)}],
 'split0_test_score': 

In [310]:
print(grid1.best_params_)

{'kernel_sizes': ((3, 3), (5, 5)), 'n_filters': (16, 32)}


In [311]:
print(grid1.best_score_)

0.55372


In [326]:
print(grid1.best_estimator_)

KerasClassifier(
	model=<function create_cnn_model at 0x0000025A408CD1F0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=128
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.25
	shuffle=True
	run_eagerly=False
	epochs=25
	n_filters=(16, 32)
	kernel_sizes=((3, 3), (5, 5))
	randseed=42
	class_weight=None
)


**Task 1.2**

Compute the final score on the test set using the *best_estimator* from the gridsearch and the `score()` method!

In [327]:
# YOUR CODE HERE
model.fit = grid1.best_estimator_
final_score1 = grid1.score(X_test,y_test)
# cnn = model
score = model.evaluate(X_test,y_test,verbose=0)
# score = cnn.evaluate(x_train.values,y_test,verbose=0)
print(final_score1)

0.60892


In [328]:
print(score)

[2.3005757331848145, 0.0835999995470047]


**Task 1.3**

Now we will focus on another "parameter" of the model. You do not have to repeat the definition of the `EarlyStopping()` and the `KerasClassifier`, we will just re-use and re-tune it. Create a **new** parameter grid called `param_grid2` which contains only the `randseed`. Set the values to run the gridsearch to 3,17 and 42!

Finally use the provided code to run the grid search. This can take a few minutes.

In [329]:
# YOUR CODE HERE
param_grid2 = dict(randseed=[317,42])
grid2 = GridSearchCV(estimator=cnn, param_grid=param_grid2,cv=2,verbose=2)
grid2.fit(X_train,y_train,callbacks=es)
grid2.cv_results_

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] END .......................................randseed=317; total time= 1.6min
[CV] END .......................................randseed=317; total time= 1.6min
[CV] END ........................................randseed=42; total time= 1.6min
[CV] END ........................................randseed=42; total time= 1.2min


{'mean_fit_time': array([94.24828851, 83.31913686]),
 'std_fit_time': array([1.52061975, 9.43214893]),
 'mean_score_time': array([1.36837876, 1.22294474]),
 'std_score_time': array([0.04294407, 0.14446521]),
 'param_randseed': masked_array(data=[317, 42],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'randseed': 317}, {'randseed': 42}],
 'split0_test_score': array([0.5716 , 0.55888]),
 'split1_test_score': array([0.58096, 0.5768 ]),
 'mean_test_score': array([0.57628, 0.56784]),
 'std_test_score': array([0.00468, 0.00896]),
 'rank_test_score': array([1, 2])}

In [330]:
print(grid2.best_params_)

{'randseed': 317}


**Task 1.4**

Compute the final score on the test set using the *best_estimator* from the gridsearch and the `score()` method and save it to the variable `final_score2`!

In [331]:
# YOUR CODE HERE
model.fit = grid2.best_estimator_
final_score2 = grid2.score(X_test,y_test)
score = model.evaluate(X_test,y_test,verbose=0)
print(final_score2)

0.63256


In [332]:
print(score)

[2.3005757331848145, 0.0835999995470047]


**Task 1.5**

Which of the following statements is true?

'a': The random state/seed is a hyper-parameter and we should find the optimal seed.

'b': Tuning the random state/seed is a questionable (reasearch) practice.

'c': Repeating experiments instead of using a single random state/seed is a questionable (research) practice.

'd': There is no randomness in the CNN defined and used in the tasks above.

*Use a variable* `x15` *and assign your answer of the question (if 'e' would be the correct answer your code should look like this:* `x15='e'`. *There is only one correct answer.*

In [ ]:
# YOUR CODE HERE
x15 = "c"
print('The answer is ' + x15 + '.')

In the remaing part of the notebook we will work with a pre-trained model. Due to storage and compute capabilites we will not use famous networks like ResNet50, but the light-weight MobileNetV2.

**Task 2.1**


Write a function `resize_to_244by244()` to change the shape of an image (represented as a tensorflow-tensor). The *MobileNetV2* requires input images with a size of $224 \times224$. Since not all images are square, there are several ways to resize a generally rectangular image to a square format. 
Implement the following methods:
1. *Simple resize*, ignoring the aspect ratio: `mode='resize'`
2. *Zero-padding*, maintaining the aspect ratio by zero padding: `mode='pad'` 
3. *Resize and central crop*, resizing maintaining the aspect ratio followed by cropping an square part of a non-square resized image: `mode='crop'`

*Hint: You might use tensorflow functions.*
[<img src="resize_demo.png" width="700"/>](resize_demo.png)

In [12]:
def resize_to_244by244(im_tf, mode):
    '''
    inputs: im_if, type: tensorflow-tensor
            mode, type: string
    outputs: im_tf, type: tensorflow-tensor        
    '''
    # YOUR CODE HERE
    img_data = tf.image.convert_image_dtype(im_tf, dtype = tf.uint8)
    if mode == 'resize':
        im_tf = tf.image.resize(img_data,[224, 224])
    if mode == 'pad':
        im_tf  = tf.image.resize_image_with_crop_or_pad(img_data,224,224)
    if mode == 'crop':
        im_tf = tf.image.central_crop(img_data,0.6)
    retype = tf.cast(im_tf, tf.float32)
    im_tf = tf.convert_to_tensor(retype)

    return im_tf

In [13]:
def jpg_file_to_mobilenet(filename=None, show=True, mode='resize'):
    im_pil = Image.open(filename)
    im_tf = img_to_array(im_pil).astype('uint8')  
    im_tf = tf.convert_to_tensor(im_tf)
    if show:
        #print('Original - Shape:', tf.shape(im_tf), ' -- NumpyType:', im_tf.numpy().dtype, ' -- Range: ', im_tf.numpy().min(), im_tf.numpy().max())
        f,a = plt.subplots(1,3,figsize=(10,5))
        f.suptitle(filename,y=.85)
        a[0].imshow(im_tf)
        a[0].set_title('Original')
    im_tf = resize_to_244by244(im_tf,mode)
    if show:
        #print('Resize - Shape:', tf.shape(im_tf), ' -- NumpyType:', im_tf.numpy().dtype, ' -- Range: ', im_tf.numpy().min(), im_tf.numpy().max())
        a[1].imshow(im_tf/255)
        a[1].set_title('Resized')
    im_tf = preprocess_input(im_tf)
    if show:
        #print('MobileNetPreporcessing - Shape:', tf.shape(im_tf), ' -- NumpyType:', im_tf.numpy().dtype, ' -- Range: ', im_tf.numpy().min(), im_tf.numpy().max())
        a[2].imshow(im_tf*0.5 + 0.5)
        a[2].set_title('MobileNetPreporcessing')
        f.tight_layout()
    im_tf = im_tf[None, ...]
    return im_tf

Keras/Tensorflow serves some prominent CNN-architectures for image recognition, like *ResNet*, *VGG* and many more. Due to its small footprint we will be using the *MobileNet v2*.

**Task 2.2**


Set up a `MobileNetV2()`-Keras model `mnv2`, pre-trained on the *ImageNet* dataset. 

In [35]:
# YOUR CODE HERE
def MobileNetv2(input_shape):
    base_model=MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
    inputs=tf.keras.layers.Input(shape=input_shape)
    x=base_model(inputs)
    x=tf.keras.layers.AveragePooling2D()(x)
    x=tf.keras.layers.Dense(1024,activation='relu')(x)
    outputs=tf.keras.layers.Dense(8,activation='softmax')(x)
    model=tf.keras.models.Model(inputs=inputs,outputs=outputs)

    plot_model(model,to_file='model.png',show_layer_names=True,show_shapes=True)

    return model
mnv2 = MobileNetv2((224, 224, 3))

**Task 2.3**

Use the function `jpg_file_to_mobilenet()` (making use of your resize function from task 2.1) to load the provided image by providing the filename and the resize mode.
Predict the output for this image and print the top-3 softmax-output values and the 3 associated classes using Keras' applications `decode_predictions()` function. Store the output of the function to the variable `out`.

AttributeError: 'list' object has no attribute 'shape'

<Figure size 720x360 with 3 Axes>